In [ ]:
import numpy as np
import matplotlib.pylab as plt
import h5py
from scipy.interpolate import RegularGridInterpolator as rgi
import matplotlib as mpl

mpl.rc('font', size=16)

c = 29979245800.
G = 6.6743e-8
msun = 1.9889e33

def setup_EOS(filename):    
    EOSfile = filename
    EOSh5 = h5py.File(EOSfile,'r')

    logtemps = EOSh5['logtemp']
    logrhos = EOSh5['logrho']
    yes = EOSh5['ye']

    table_energy = EOSh5['logenergy'][:,:,:]
    mif_logenergy = rgi((yes,logtemps,logrhos), table_energy)
    table_logpressure = EOSh5['logpress'][:,:,:]
    mif_logpressure = (rgi((yes,logtemps,logrhos), table_logpressure))
    table_munu = EOSh5['munu'][:,:,:]
    mif_munu = rgi((yes,logtemps,logrhos), table_munu)

    energy_shift = EOSh5['energy_shift'][0]
    targetT = np.log10(0.1)

    EOS = {'press':np.zeros(len(logrhos)),'energy':np.zeros(len(logrhos)),'ye':np.zeros(len(logrhos)),'logrhos':logrhos}
    Tindex = np.argmin(np.abs(np.asarray(logtemps)-targetT))

    rhoindex = 0
    for logrho in logrhos:
        #first find ye that gives beta equilibrium
        mus = table_munu[:,Tindex,rhoindex]
        EOS['ye'][rhoindex] = np.interp(0.0,mus,yes)
        EOS['press'][rhoindex] = 10.0**(mif_logpressure(np.array([EOS['ye'][rhoindex],targetT,logrho]).T)[0])
        EOS['energy'][rhoindex] = 10.0**(mif_logenergy(np.array([EOS['ye'][rhoindex],targetT,logrho]).T)[0])-energy_shift
        rhoindex += 1

    return EOS
        
EOSDD2 = setup_EOS("DD2.h5")
EOSSFHo = setup_EOS("SFHo.h5")

In [ ]:
def pressure_from_rho(rho,EOS):
    return np.interp(np.log10(rho),EOS['logrhos'],EOS['press'])

def ye_from_rho(rho,EOS):
    return np.interp(np.log10(rho),EOS['logrhos'],EOS['ye'])

def rho_from_pressure(pressure,EOS):
    return 10.0**(np.interp(pressure,EOS['press'],EOS['logrhos']))

def epsilon_from_rho(rho,EOS):
    return np.interp(np.log10(rho),EOS['logrhos'],EOS['energy'])

def rhoR_from_rho(rho,EOS):
    return rho*(1.0+epsilon_from_rho(rho,EOS)/c**2)

def dPdr(rhoR,mass,pressure,radius):
    if radius==0.0: 
        dPdr = 0.0
    else:    
        dPdr = (-G*rhoR/radius**2 *
            (1.0 + pressure/(rhoR*c**2)) *
            (mass + 4.0*np.pi*radius**3*pressure/c**2) /
            (1.0 - 2.0*G*mass/(radius*c**2)))
    
    return dPdr
    
def dmdr(rhoR,radius):
    return 4.0*np.pi*radius**2*rhoR

def dmbdr(rho,mass,radius):
    if (mass==0.0):
        return 4.0*np.pi*radius**2*rho
    else:
        return 4.0*np.pi*radius**2*rho*(1.0-2.0*G*mass/(radius*c**2))**(-0.5)



def solve_tov(EOS,centralrho=1e15,nzones=1200,deltar=1500.):
    rho0 = centralrho
    P0 = pressure_from_rho(rho0,EOS)
    
    pressure = np.zeros(nzones)
    rho = np.zeros(nzones)
    ye = np.zeros(nzones)
    mass = np.zeros(nzones)
    bmass = np.zeros(nzones)
    radius = np.zeros(nzones)

    #initial conditions
    pressure[0] = P0
    rho[0] = rho0
    ye[0] = ye_from_rho(rho0,EOS)

    endi = nzones
    for i in range(1,nzones):
        rhoR = rhoR_from_rho(rho[i-1],EOS)
        radius[i] = radius[i-1]+deltar
        mass[i] = mass[i-1] + dmdr(rhoR,radius[i-1])*deltar
        bmass[i] = bmass[i-1] + dmbdr(rho[i-1],mass[i-1],radius[i-1])*deltar
        newpress = pressure[i-1] + dPdr(rhoR,mass[i-1],pressure[i-1],radius[i-1])*deltar
        if newpress < 1e25: 
            endi = i
            break
        pressure[i] = newpress
        rho[i] = rho_from_pressure(pressure[i],EOS)
        ye[i] = ye_from_rho(rho[i],EOS)
        
    return radius[:endi],mass[:endi],rho[:endi],pressure[:endi],ye[:endi],bmass[:endi]

In [ ]:
centralrho = 1e15
data = solve_tov(EOSDD2,centralrho,nzones=1200,deltar=1500.)
plt.plot(data[0]/1e5,data[1]/msun)
plt.ylabel("Mass [$M_\odot$]")
plt.xlabel("Radius [km]")
plt.show()

plt.plot(data[0]/1e5,data[2])
plt.ylabel("Density [g/cm3]")
plt.xlabel("Radius [km]")
plt.yscale('log')
plt.show()

plt.plot(data[0]/1e5,data[3])
plt.ylabel("Pressure [erg/cm3]")
plt.xlabel("Radius [km]")
plt.yscale('log')
plt.show()

plt.plot(data[0]/1e5,data[4])
plt.ylabel("Ye")
plt.xlabel("Radius [km]")
plt.show()

In [ ]:
minrho = 2e14
maxrho = 3e15
steps = 20
star = {'rad':np.zeros(steps),'mass':np.zeros(steps),'bmass':np.zeros(steps),'centralrho':np.zeros(steps)}
for i in range(steps):
    centralrho = 10.0**(np.log10(minrho) + (np.log10(maxrho)-np.log10(minrho))*i/(steps-1))
    data = solve_tov(EOSDD2,centralrho,nzones=1200,deltar=1500.)
    star['centralrho'][i] = centralrho
    star['rad'][i] = data[0][-1] 
    star['mass'][i] = data[1][-1]
    star['bmass'][i] = data[5][-1]
star_DD2 = star

star = {'rad':np.zeros(steps),'mass':np.zeros(steps),'bmass':np.zeros(steps),'centralrho':np.zeros(steps)}
for i in range(steps):
    centralrho = 10.0**(np.log10(minrho) + (np.log10(maxrho)-np.log10(minrho))*i/(steps-1))
    data = solve_tov(EOSSFHo,centralrho,nzones=1200,deltar=1500.)
    star['centralrho'][i] = centralrho
    star['rad'][i] = data[0][-1] 
    star['mass'][i] = data[1][-1]
    star['bmass'][i] = data[5][-1]
star_SFHo = star    


In [ ]:
plt.plot(star_DD2['rad']/1e5,star_DD2['mass']/1.9889e33,'C0--',label="DD2 MGrav")
plt.plot(star_DD2['rad']/1e5,star_DD2['bmass']/1.9889e33,'C1--',label="DD2 MBary")
plt.plot(star_SFHo['rad']/1e5,star_SFHo['mass']/1.9889e33,'C0-',label="SFHo MGrav")
plt.plot(star_SFHo['rad']/1e5,star_SFHo['bmass']/1.9889e33,'C1-',label="SFHo MBary")
plt.ylabel("Mass [$M_\odot$]")
plt.xlabel("Radius [km]")
plt.legend()
plt.show()